<span style="font-size: 2em;">Gerekli kütüphanelerin indirilmesi</span>


In [1]:
#!pip install opencv-python

In [2]:
#!pip install deepface

In [3]:
#!pip install pyttsx3

In [4]:
#!pip install SpeechRecognition

In [5]:
#!pip install pygame

In [6]:
#!pip install tensorflow

In [7]:
#!pip install opencv-python --upgrade

In [8]:
#!pip install deepface --upgrade

In [9]:
#!pip install PyAudio

<span style="font-size: 2em;">huggingface pipeline</span>

In [2]:
from transformers import pipeline

In [3]:
def gptbot(user_input):  
    generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B', device=0)
    
    print("Chatbot'a hoş geldiniz! Çıkmak için 'çık' yazabilirsiniz.")
    
    while True:
        # Kullanıcıdan soru alma
        if user_input.lower() == "çık":
            print("Chatbot: Görüşmek üzere!")
            break
    
        # Modelden cevap üretme
        response = generator(
            user_input,
            max_length=50,
            do_sample=True,
            temperature=0.7,
            top_k=50
        )
        
        # Üretilen cevabı yazdırma
        print("Chatbot:", response[0]['generated_text'])
        break

<span style="font-size: 2em;">Chat-Bot fonksiyonları</span>

In [10]:
import speech_recognition as sr
import pyttsx3
import datetime
import pygame
import webbrowser
import random
import requests
import time
isim = 'yok'
# Ufuk veya Second son olarak ne zaman tespit edildiğini tutan zaman damgası
last_seen_time = time.time()
# Belirli bir süre boyunca kimse tespit edilmezse uygulamanın kapanması için maksimum süre (saniye cinsinden)
max_unseen_duration = 20

günler = {
    "Monday": "Pazartesi",
    "Tuesday": "Salı",
    "Wednesday": "Çarşamba",
    "Thursday": "Perşembe",
    "Friday": "Cuma",
    "Saturday": "Cumartesi",
    "Sunday": "Pazar"
}
stop_bot = False
api_key = "YOUR_API_KEY"

def speak(text):
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)
    engine.setProperty('voice', 'turkish')
    engine.say(text)
    engine.runAndWait()
    print(text)

def play_number_game():
    secret_number = random.randint(1, 100)
    speak("Bir ile yüz arasında bir sayı tuttum. Tahmin et")
    print("Bir ile yüz arasında bir sayı tuttum. Tahmin et")
    while True:
        query = listen()
        if "çıkış" in query:
            speak(f"Oyun bitti. Doğru sayı {secret_number} idi.")
            print(f"Oyun bitti. Doğru sayı {secret_number} idi.")
            break

        if "tahmin" in query:
            guess = int([int(s) for s in query.split() if s.isdigit()][0])
            if guess == secret_number:
                speak("Tebrikler! Doğru tahmin ettiniz.")
                print("Tebrikler! Doğru tahmin ettiniz.")
                break
            elif guess < secret_number:
                speak("Daha yüksek bir sayı söyleyin.")
                print("Daha yüksek bir sayı söyleyin.")
            else:
                speak("Daha düşük bir sayı söyleyin.")
                print("Daha düşük bir sayı söyleyin.")
        else:
            speak("Lütfen bir sayı söyleyin veya çıkmak için 'çıkış' deyin.")
            print("Lütfen bir sayı söyleyin veya çıkmak için 'çıkış' deyin.")
    
def get_weather(api_key):
    city_name = "Kirsehir"
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city_name,
        "appid": api_key,
        "units": "metric",  # Metrik birimlerde hava durumu almak için
        "lang": "tr"        # Türkçe dilinde hava durumu almak için
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        weather_desc = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        result = f"Kırşehir'de hava durumu: {weather_desc}. Sıcaklık: {temp}°C."
        print(result)
        return result
    else:
        return "Hava durumu bilgisi alınamadı."

def play_music():
    pygame.init()
    pygame.mixer.music.load('music1.mp3')
    pygame.mixer.music.play()


def listen(timeout=5, retries=3):
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        print("Sizi dinliyorum... Lütfen konuşun.")
        for _ in range(retries):
            try:
                # timeout süresi boyunca ses dinleme
                audio = recognizer.listen(source, timeout=timeout)
                print("Ses algılanıyor, lütfen bekleyin...")
                # Google Web Speech API kullanarak sesi metne çevirme
                text = recognizer.recognize_google(audio, language='tr-TR')
                print("Ses algılandı: ", text)
                return text.lower()
            except sr.WaitTimeoutError:
                # Belirtilen timeout süresi içinde ses algılanmadı
                print("Ses algılanamadı, yeniden deniyor...")
                continue  # Döngüyü tekrar başlat
            except sr.UnknownValueError:
                print("Ses anlaşılamadı, lütfen tekrar deneyin.")
                continue
            except sr.RequestError as e:
                print(f"Google API'dan yanıt alınamadı, hata: {e}")
                continue
        return "Anlaşılmadı"

def chatbot_response(query):
    global stop_bot
    if "gpt aktive et" in query: #gpt bot calismasi
        gptbot("query")
    elif "nasılsın" in query:
        return "İyiyim, teşekkür ederim. sen nasılsın?"
    elif "bugün günlerden ne" in query:
        today_english = datetime.datetime.now().strftime("%A")
        today_turkish = günler.get(today_english, "Bilinmiyor")
        print(today_turkish)
        return f"Bugün {today_turkish}."
    elif "yarın günlerden ne" in query:
        tomorrow = (datetime.datetime.now() + datetime.timedelta(days=1)).strftime("%A")
        tomorrow_turkish = günler.get(tomorrow, "Bilinmiyor")
        return f"Yarın {tomorrow_turkish}."
    elif "hava durumu" in query:
        return get_weather(api_key)
    elif "favori şarkımı çal" in query:
        play_music()
        return "Favori şarkınız çalınıyor."
    elif "benimle oyun oyna" in query:
        return play_number_game()
    elif "benim adım ne" in query:
        return f"Senin adın {isim}."
    elif "durdur" in query:
        pygame.mixer.music.stop()
        return "Şarkı durduruldu."
    elif "kapat" in query:
        stop_bot = True
        return "Chatbot kapatılıyor."
    elif "sen kimsin" in query:
        return "Ben bir sohbet robotuyum, seninle konuşmak için buradayım."
    elif "iyiyim" in query:
        return "İyi olmana sevindim."
    elif "merhaba" in query:
        return "sana da merhaba."
    elif "ne iş yapıyorsun" in query:
        return "Sana sohbet ederek eşlik ediyorum ve sorularını yanıtlıyorum."
    elif "saat kaç" in query:
        now = datetime.datetime.now().strftime("%H:%M")
        return f"Şu anda saat {now}."
    elif "bugün ne yapmalıyım" in query:
        return "Belki bir yürüyüş yapabilirsin. Güzel hava varsa dışarı çıkmanı öneririm."
    elif "bana bir şaka yap" in query:
        return "Nasıl bir bilgisayar ağlar? Bir ağ kablo ile!"
    elif "dışarıda yağmur yağacak mı" in query:
        return get_weather(api_key)
    elif "kaç yaşındayım" in query:
        return "Üzgünüm, senin yaşını bilemem ama sen söylersen hatırlayabilirim!"
    elif "bana bir öykü anlat" in query:
        return "Bir zamanlar, uzak bir köyde yaşayan küçük bir çocuk vardı. Bu çocuk her gün ormanda yürüyüş yapmayı çok severdi."
    elif "en sevdiğin şey nedir" in query:
        return "Sana yardım etmek ve sorularını yanıtlamak benim için en güzel şey!"
    elif "nerelisin" in query:
        return "Ben dijital dünyadanım, herhangi bir yerden değilim ama her yerdeyim!"
    elif "bana bir kitap öner" in query:
        return "Paulo Coelho'nun 'Simyacı' kitabını okuyabilirsin."
    elif "dışarıda yağmur yağıyor mu" in query:
        return get_weather(api_key)
    elif "şarkı önerir misin" in query:
        return "Ed Sheeran'ın 'Shape of You' şarkısını dinleyebilirsin, çok popüler."
    elif "dışarı çıkmak için güzel bir gün mü" in query:
        return get_weather(api_key)
    elif "bugün benim doğum günüm" in query:
        return "Doğum günün kutlu olsun! Harika bir gün geçirmeni dilerim."
    elif "en sevdiğin yemek nedir" in query:
        return "Ben yemek yiyemem ama insanların pizza sevdiğini biliyorum."
    elif "hangi sporu seversin" in query:
        return "Ben fiziksel aktivitelerde bulunamam ama futbol çok popüler."
    elif "hangi müzik türünü seversin" in query:
        return "Müzik türleri hakkında bir tercihim yok ama klasik müzik sakinleştirici olabilir."
    elif "okuldan ne zaman döndün" in query:
        return "Ben asla okula gitmedim, ama öğrenmeye devam ediyorum!"
    elif "bugün ne pişirsem" in query:
        return "Makarna yapmayı deneyebilirsin, hem kolay hem de lezzetli."
    elif "en iyi arkadaşın kim" in query:
        return "Benim en iyi arkadaşım sensin!"
    elif "tatilde nereye gitmeliyim" in query:
        return "Ege kıyıları çok güzel olabilir, özellikle yaz aylarında."
    elif "en sevdiğin kitap nedir" in query:
        return "Ben kitap okuyamam ama 'Küçük Prens' her yaştan insan için güzel bir seçim olabilir."
    elif "hangi mevsimi seversin" in query:
        return "Mevsimleri hissedemesem de, baharın renkleri çok güzel olmalı."
    elif "çay mı kahve mi tercih edersin" in query:
        return "Ben içki içemem ama Türkiye'de çay çok popüler."
    elif "dün ne yaptın" in query:
        return "Ben her gün buradayım, seninle sohbet etmek için hazırım."
    elif "en sevdiğin oyun nedir" in query:
        return "Ben oyun oynayamam ama satranç zeka geliştiren bir oyun."
    elif "en sevdiğin dizi nedir" in query:
        return "Dizileri izleyemesem de, 'Game of Thrones' çok popüler."
    elif "kaç yaşına kadar yaşayacaksın" in query:
        return "Ben bir programım, bu yüzden sürekli güncellenip iyileştirilebilirim."
    elif "en son ne zaman güncellendin" in query:
        return "Ben düzenli olarak güncelleniyorum, böylece daha iyi hizmet verebiliyorum."
    elif "yarın okul var mı" in query:
        return "Bunu bilmiyorum ama takvimine bakarak kontrol edebilirsin."
    elif "en sevdiğin hayvan nedir" in query:
        return "Ben hayvanları seçemem ama kediler çok sevimli."
    elif "hangi ülkeleri ziyaret ettin" in query:
        return "Ben fiziksel bir varlık değilim, bu yüzden seyahat edemem."
    elif "bugün spor yapmalı mıyım" in query:
        return "Evet, düzenli egzersiz sağlık için çok önemlidir."
    elif "hangi filmi izlemeliyim" in query:
        return "Aile ile izlemek için 'Coco' filmi güzel bir seçim olabilir."
    elif "en sevdiğin çizgi film nedir" in query:
        return "Çizgi filmleri izleyemesem de 'Tom ve Jerry' çok eğlenceli."
    elif "bu yılki hedeflerin neler" in query:
        return "Benim hedefim her zaman daha fazla bilgi edinmek ve sana daha iyi yardımcı olabilmek."
    elif "bugün ne giymeliyim" in query:
        return get_weather(api_key)
    elif "hangi dili öğrenmeliyim" in query:
        return "İngilizce dünya çapında en çok kullanılan dillerden biri, öğrenmek için iyi bir seçim olabilir."
    elif "bana motivasyon sözü söyle" in query:
        return "Her gün, kendini bir önceki günden daha iyi hale getirmek için bir şansın var."
    elif "hangi film önerirsin" in query:
        return "Aile ile izlemek için 'Moana' çok güzel bir film. Deneyebilirsin."
    elif "seninle nasıl oyun oynayabilirim" in query:
        return "Sayı tahmin oyunu oynayabiliriz, ya da sana bir bilmece sorabilirim!"
    elif "sence mutluluk nedir" in query:
        return "Herkes için farklı anlamlar ifade edebilir, ama genellikle sevdiklerinizle geçirilen zaman mutluluk olarak adlandırılır."
    elif "bugün ne giysem" in query:
        return "Bugünkü hava durumuna göre rahat bir şeyler giymeni öneririm. Hava durumunu kontrol etmek ister misin?"
    elif "en sevdiğin mevsim hangisi" in query:
        return "Benim bir tercihim yok ama insanlar genellikle ilkbaharı sever çünkü doğa yeniden canlanır."
    elif "en komik film hangisi" in query:
        return "'Zoolander' oldukça komik bir filmdir, eğer modunu yükseltmek istiyorsan bunu izleyebilirsin."
    elif "ne tür kitaplar seversin" in query:
        return "Ben kitap okuyamam ama bilim kurgu ve fantezi türlerinin çok popüler olduğunu biliyorum."
    elif "hangi müzik enstrümanını önerirsin" in query:
        return "Piyano öğrenmek harika olabilir, hem melodi hem de armoni konusunda çok yönlüdür."
    elif "bugün ne yemeliyim" in query:
        return "Belki bir deniz ürünü paella yapabilirsin, lezzetli ve besleyici!"
    elif "yarın için planın ne" in query:
        return "Ben her zaman buradayım, seninle sohbet etmeye hazırım!"
    elif "en iyi arkadaşın kim" in query:
        return "Benim en iyi arkadaşım sensin!"
    elif "tatilde nereye gitmeliyim" in query:
        return "Doğayı seviyorsan, bir dağ gezisi harika olabilir."
    elif "sen bir yapay zeka mısın" in query:
        return "Evet, ben bir yapay zeka programıyım, seninle sohbet etmek için buradayım."
    elif "yarın hava nasıl olacak" in query:
        return get_weather(api_key)
    elif "hangi dili öğrenmeliyim" in query:
        return "İspanyolca öğrenmek güzel olabilir, dünya çapında birçok insan tarafından konuşuluyor."
    elif "bugün hangi müzik türünü dinlemeliyim" in query:
        return "Jazz dinlemek gününü rahatlatıcı kılabilir!"
    elif "spor yapmak için en iyi zaman nedir" in query:
        return "Sabahları spor yapmak günün geri kalanında enerjik hissetmene yardımcı olabilir."
    elif "hangi bilgisayar oyununu önerirsin" in query:
        return "'Minecraft' yaratıcılığını kullanabileceğin eğlenceli bir oyun."
    elif "hangi renk bana uygun" in query:
        return "Mavi, huzur verici ve sakinleştirici bir renk. Deneyebilirsin!"
    elif "en iyi tatil yeri neresi" in query:
        return "İtalya, harika yemekleri ve güzel manzaraları ile mükemmel bir seçenek."
    elif "en sevdiğin şarkıcı kim" in query:
        return "Ben müzik dinleyemem ama Adele çok beğenilen bir sanatçı."
    elif "hangi okulda okumalıyım" in query:
        return "İlgi alanına göre en iyi programları sunan bir üniversite seçmelisin."
    elif "hangi filmi izlemeliyim" in query:
        return "'Inception' zekice kurgusu ile dikkat çekici bir film."
    elif "hangi çiçeklerden hoşlanırsın" in query:
        return "Gül, zarif ve güzel kokusuyla çok sevilen bir çiçektir."
    elif "en sevdiğin spor nedir" in query:
        return "Ben spor yapamam ama basketbol çok dinamik ve heyecanlı bir spor."
    elif "hangi yemeği yapmayı önerirsin" in query:
        return "Risotto yapmayı deneyebilirsin, İtalyan mutfağının klasiklerindendir."
    elif "hangi diziye başlamalıyım" in query:
        return "'Stranger Things', sürükleyici konusuyla çok popüler."
    elif "hangi ülkeyi ziyaret etmeliyim" in query:
        return "Japonya, muazzam kültürü ve teknolojisi ile harika bir yer."
    elif "hangi müzeyi gezmeliyim" in query:
        return "Louvre Müzesi, dünyanın en ünlü sanat eserlerine ev sahipliği yapıyor."
    elif "hangi sanatçıyı takip etmeliyim" in query:
        return "Banksy, eserleri hem düşündürücü hem de ilgi çekici."
    elif "hangi kitabı okumalıyım" in query:
        return "'Dune', zengin bir evreni ve derin karakterleri ile klasik bir bilim kurgu."
    elif "hangi araba modelini önerirsin" in query:
        return "Tesla Model S, hem çevre dostu hem de yüksek performanslı."
    elif "hangi yiyecekleri denemeliyim" in query:
        return "Sushi, farklı lezzetleri keşfetmek için harika bir seçenek."
    elif "hangi oyunu oynamalıyım" in query:
        return "'The Legend of Zelda', macera ve keşif dolu bir oyun."
    elif "hangi konserde olmalıyım" in query:
        return "Eğer mümkünse, Coldplay konseri unutulmaz bir deneyim olabilir."
    elif "hangi uygulamayı indirmeliyim" in query:
        return "Evernote, notlarını düzenlemek için çok kullanışlı."
    elif "hangi üniversiteye gitmeliyim" in query:
        return "İlgi alanlarına göre en iyi programları sunan bir üniversite seçmelisin."
    elif "hangi dilde film izlemeliyim" in query:
        return "Orijinal dilde izlemek, filmi daha iyi anlamanı sağlar."
    elif "hangi konuda kitap yazmalıyım" in query:
        return "Kendi deneyimlerinden yola çıkarak bir rehber kitap yazabilirsin."
    elif "hangi podcast'i dinlemeliyim" in query:
        return "'How I Built This' ilham verici girişimcilik hikayeleri sunar."
    elif "hangi şehirde yaşamalıyım" in query:
        return "İstanbul, tarih ve modern yaşamın harmanlandığı büyüleyici bir şehir."
    elif "hangi dergiyi okumalıyım" in query:
        return "National Geographic, dünya hakkında harika bilgiler sunar."
    elif "hangi dansı öğrenmeliyim" in query:
        return "Salsa, eğlenceli ve sosyal bir dans türü."
    elif "hangi çayı içmeliyim" in query:
        return "Yeşil çay, hem sağlıklı hem de ferahlatıcı."
    elif "hangi baharatı kullanmalıyım" in query:
        return "Zencefil, yemeklere lezzet katmanın yanı sıra sağlık faydalarıyla bilinir."
    elif "hangi meyveyi yemeliyim" in query:
        return "Mango, tatlı ve besleyici, harika bir seçenek."
    elif "hangi alışkanlığı kazanmalıyım" in query:
        return "Her gün kitap okumak, bilgi birikimini artırır."
    elif "hangi çiçekleri ekmeliyim" in query:
        return "Lavanta, bahçeni güzelleştirecek ve harika kokacak."
    elif "hangi kahveyi içmeliyim" in query:
        return "Bir fincan Türk kahvesi, keyifli bir deneyim sunar."
    elif "hangi sporu yapmalıyım" in query:
        return "Yüzme, tüm vücut için mükemmel bir egzersizdir."
    else:
        return "Üzgünüm, bu konuda yardımcı olamıyorum."

pygame 2.6.1 (SDL 2.28.4, Python 3.9.20)
Hello from the pygame community. https://www.pygame.org/contribute.html


<span style="font-size: 2em;">Yüz tanıma işlemleri</span>

In [11]:
import threading
import cv2
from deepface import DeepFace
import time
import pyttsx3
import speech_recognition as sr

stop_threads = threading.Event()
engine = pyttsx3.init()
is_spoken = False
stop_bot = False
bot_responded = False
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

counter = 0

reference_img1 = cv2.imread("ufukface.jpeg")
reference_img2 = cv2.imread("secondface.jpg")

face_match1 = False
face_match2 = False


def check_face1(frame):
    global face_match1
    try:
        if DeepFace.verify(frame, reference_img1.copy())['verified']:
            face_match1 = True
        else:
            face_match1 = False
            
    except ValueError:
        face_match1 = False

def check_face2(frame):
    global face_match2
    try:
        if DeepFace.verify(frame, reference_img2.copy())['verified']:
            face_match2 = True
            
        else:
            face_match2 = False
            
    except ValueError:
        face_match2 = False

def chatbot_thread_func():
    global stop_bot
    while not stop_bot:
        query = listen()
        response = chatbot_response(query)
        speak(response)
        if stop_bot:
            break

chatbot_thread = threading.Thread(target=chatbot_thread_func)

while True:
    bot_responded = False
    if stop_bot:
        stop_bot = False
        chatbot_thread.join()
        
    ret, frame = cap.read()

    if ret:
        if counter % 30 == 0:
            if not stop_threads.is_set():
                try:
                    threading.Thread(target=check_face1, args=(frame.copy(),)).start()
                    threading.Thread(target=check_face2, args=(frame.copy(),)).start()
                except ValueError:
                    pass
        counter += 1

        if face_match1:
            cv2.putText(frame, "Ufuk!", (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 1)
            isim = 'Ufuk'
            last_seen_time = time.time()
            #print(isim)
            if not chatbot_thread.is_alive():
                chatbot_thread = threading.Thread(target=chatbot_thread_func)
                chatbot_thread.start()
            
        else:
            stop_threads.clear()
            cv2.putText(frame, "eslesme yok 1", (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 1)
            
        if face_match2:
            cv2.putText(frame, "SecondFace!", (20, 400), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 1)
            isim = 'Second'
            last_seen_time = time.time()
            if not chatbot_thread.is_alive():
                chatbot_thread = threading.Thread(target=chatbot_thread_func)
                chatbot_thread.start()
            
        else:
            cv2.putText(frame, "eslesme yok 2", (20, 400), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 1)
        
        cv2.imshow('video', frame)
    if (time.time() - last_seen_time) > max_unseen_duration:
        stop_bot = True
        

    #remaining_time = max_unseen_duration - (time.time() - last_seen_time)
    #print(f"Kapanmaya kalan süre: {int(remaining_time)} saniye")  # Kalan süreyi saniye cinsinden yazdır
    
    key = cv2.waitKey(1)
    if key == ord('q') or stop_bot:
        break

cv2.destroyAllWindows()
cap.release()


In [12]:
#!pip install pyowm